[1. Two Sum](#two_sum)

[121. Best Time to Buy and Sell Stock](#stocks)

[238. Product of Array Except Self](#product_of_array_except_self)

[53. Maximum Subarray](#max_sum_subarray)

[152. Maximum Product Subarray](#max_prod_subarray)

[33. Search in Rotated Sorted Array](#search_rotated_sorted_array)

[15. 3Sum](#3_sum)

# 1. Two Sum
<a id='two_sum'></a>
https://leetcode.com/problems/two-sum/

intuition: https://www.youtube.com/watch?v=KLlXCFG5TnA

you can always brute force but you can one pass by:
- Using hashmap to store target-cur_val as key in dct. This way when the matching number comes you have old index and current.

time: O(n) one pass

space: O(n) can potentially add everything into hashmap


In [1]:
def twoSum(nums, target):
    prevMap = {}
    for i, num in enumerate(nums):
        if num in prevMap:
            return [prevMap[num], i]
        prevMap[target - num] = i

In [2]:
twoSum([3,2,4,8,5], 9)

[2, 4]

# 121. Best Time to Buy and Sell Stock
<a id='stocks'></a>

https://leetcode.com/problems/best-time-to-buy-and-sell-stock/

intuition: https://www.youtube.com/watch?v=1pkOgXD63yU

since need to sell stock for a profit, meaning right pointer always need to be greater than left, otherwise its negative. This means:

1. To make sure always buy low, left pointer will always be lower than right. Update left to be right if right is lower. This will ensure left is always local min.
2. To make sure max profit, keep track of it at every single iteration, so case like [2,8,1,3] won't be an edge case if only check max profit at the end.
3. if right pointer is not out of bound (also using condition 1, left is always lower than right), move right pointer right.

time: O(n) one pass

space: O(1) because nothing is really stored

How I think the problem is figured out:

1. You need two pointers to track low and high.
2. Similar to brute force with two for loop, you fix left pointer and iteratively check which right has highest value.
3. But without any modification, that's just brute force, so we observe when right pointer is lower than left, we need to update left to right pointer. This may seem like skipping value between L and R, but we did not. All the values between L and R have already been covered from R pointer moving to the right up until that point, so move L to R pointer is just fine.

In [3]:
def maxProfit(prices):
    l,r = 0, 1
    max_p = 0
    while r < len(prices):
        if prices[l] < prices[r]:
            max_p = max(max_p, prices[r] - prices[l])
        else: #if l == r, l will also be updated to r
            l = r
        r += 1 # add r after eveyrthing, otherwise r will move too early
    return max_p

In [4]:
maxProfit([2,8,1,6])

6

# 238. Product of Array Except Self
<a id='product_of_array_except_self'></a>

https://leetcode.com/problems/product-of-array-except-self/


figured out by myself!

cannot use division, need O(n) time complexity and O(1) space complexity (array output doesn't count toward space complexity)

1. notice each index needs to be all prod of every other indexes
2. notice not ordered and need combination of all other indexes (so not binary search). Also notice cannot sort it since it will be O(nlogn) and cannot use any data structure since itll break O(1) space complexity. Realize only for loop, but how?
3. notice each index is made of index's left side nums multipled and right side nums multipled. realize two for loop for forward and backward multipled will do.

In [5]:
def productExceptSelf(nums):
    output = [1]*len(nums)
    cur = 1
    for i in range(len(nums) - 1):
        cur *= nums[i]
        output[i + 1] = cur
    cur = 1
    for i in range(len(nums) - 1, 0, -1):
        cur *= nums[i]
        output[i - 1] *= cur
    return output

In [6]:
productExceptSelf([2,3,4,5,6])

[360, 240, 180, 144, 120]

# 53. Maximum Subarray
<a id='max_sum_subarray'></a>

https://leetcode.com/problems/maximum-subarray/

intuition: https://www.youtube.com/watch?v=5WZl3MMT0Eg

1. notice O(n^3) time complexity solution, three for loop: one for start to finish we call i (beginning), one from i to finish we call j (end), one for loop to calculate sum of i to j. keep track of max
2. notice O(n^2) time complexity solution, no need for the third for loop of calculating sum of i to j every single time, just add the next value of previous sum. example: [1,2,3,4], we are at index 2 with sum of [1,2,3], now we just add 4 instead of all four again.
3. notice O(n) time complexity solution, when we add numbers, we dont need to keep track of PREVIOUSLY seen summed negative value that we know will ONLY decrease the maximum possible value, so we just keep track of indexes that will only increase the value. Keep track of max value for every iter! Since there may be big single values that will be forgotten, and they alone are the max subArray.

what about values that are after current index that can make value negative? We need to keep track since we do not know if there is a big value later on that will make it greater than before.

example: [1,-2,3,4]
- from first iteration, currentSum value 1 max 1
- second iter, sum value -1 max 1
- third iter, we notice the values before 3 will only decrease it so we wipe currentSum to be 0 again and update max value to 3
- fourth iter, sum value 7, max 7

fyi: kadane's algo can solve this

In [19]:
def maxSubArray(nums):
    max_sub = nums[0]
    cur_sum = 0
    
    for n in nums:
        if cur_sum < 0:
            cur_sum = 0
        cur_sum += n
        max_sub = max(max_sub, cur_sum)
    return max_sub

In [20]:
maxSubArray([100,-101,1,2,3,4,5,6]) # good example of why max needs to be tracked constantly

100

# 152. Maximum Product Subarray
<a id='max_prod_subarray'></a>

https://leetcode.com/problems/maximum-product-subarray/

intuition: https://www.youtube.com/watch?v=lXVy6YWFcRM&t=50s

1. notice O(n^2) solution, loop through two times and keep track of max at every iter.
2. problem has three case: positive, negative and zero
- positive: total multiplied will get bigger no matter what
- negative: This is the key of the problem. We do not know how many negatives there are to make it positive or negative, so using the idea of max and min value at every iter will keep track of both cases: positive case and negative case.
- zero: no need to worry about it, since we keep track of max and min with the current index into consideration as well.

time complexity: O(n)

space complexity: O(1)

In [23]:
def maxProduct(nums):
    cur_max, cur_min = 1, 1
    max_val = max(nums)
    #max, min for each index because there is negative to keep track of
    for num in nums:
        tmp = cur_max*num #need this here so won't override cur_max when inside cur_min declaration
        cur_max = max(tmp, cur_min*num, num) # compare all possible value at each iter: index itself, i*max, and i*min (may result in pos)
        cur_min = min(tmp, cur_min*num, num)
        max_val = max(max_val, cur_max)
    return max_val

In [24]:
maxProduct([1,2,-100,0,3])

3

# 33. Search in Rotated Sorted Array
<a id='search_rotated_sorted_array'></a>

https://leetcode.com/problems/search-in-rotated-sorted-array/

figured out by myself first try!

intuition:
- search sorted array will likely be binary search.
- rotate or not, its still ordered, just need to take pivot into account
- only three cases: no rotation, pivot right, and pivot left

1. check if one side is pivotted (I did right side) and see if target is in pivot, if not move right pointer to mid, else left to mid + 1
2. if not pivotted, check target is in right side or not, move pointer like above
3. pointers will eventually converge to one index

time comlexity: O(logn)

space complexity: O(1)

In [37]:
def search(nums, target):
    l,r = 0, len(nums) - 1
    while l < r:
        mid = (l+ r)//2
        
        #early return to save compute
        if nums[l] == target:
            return l
        if nums[r] == target:
            return r
        if nums[mid] == target:
            return mid
        
        #check if right is pivot
        if nums[mid] > nums[r]:
            if (nums[mid] < target) or (target <= nums[r]):
                l = mid + 1
            else:
                r = mid
        #if not pivot, right is ordered and left can either be pivot or not, use else since left checking cond is already covered within
        else:
            if nums[mid] < target and target < nums[r]:
                l = mid + 1
            else:
                r = mid
    
    if nums[l] != target:
        return -1
    return l

In [38]:
search([9,10,11,12,1,3,5,6,7,8], 12)

3

# 15. 3Sum
<a id='3_sum'></a>
https://leetcode.com/problems/3sum/

intuition: https://www.youtube.com/watch?v=jzZsG8n2R9A

my failed approach (IMPORTANT):
- sort nums for left and right comparison
- thought I can start l = 0 r = len(nums) - 1 and move mid pointer to find if sum to zero, then shift either left or right pointer (depending if left is greater magnitude than right or vice versa), then update mid to the right of left.
    - this causes some edge case: example [-4,-3,-1,0,4], if left and right have same magnitude, how would you know which to shift? you don't.
- FYI: there are multiple combination that can form to zero, assuming first -1 is fixed [-1,0,1], [-1,1,2]

correct approach:
- notice if i don't sort, we cannot avoid extra memory usage to keep track of duplicates (and its difficult to track since can't really design a hashmap of lists in short time), so sort. example[-3,2,1,-3]
- notice idea of "solution set must not contain duplicate triplets" is combination, and in combination if 1 number is guaranteed to be unique at each search, then no duplciates will occur.
1. sort
2. for loop up to third to last
3. at each iteration calculate two sum to see if it sums up to 0

In [66]:
def threeSum(nums):
    res = []
    nums.sort()

    for i, a in enumerate(nums):
        if i > 0 and a == nums[i - 1]:
            continue
        l, r = i + 1, len(nums) - 1
        while l < r: 
            threeSum = a + nums[l] + nums[r]
            if threeSum > 0:
                r -= 1
            elif threeSum < 0:
                l += 1
            else:
                res += [[a, nums[l], nums[r]]]
                l += 1
                # elegant update:
                # only need to continuous update here, because while loop above will take care of r and l shifts due to unique solution
                # can only show up once. ex: [-2,-2,0,0,2,2], found [-2,0,2], if continously update left until left is at 0, notice
                # right will move in beacuse sum will always be greater than 0 when 0,0,2 or 0,2,2
                while nums[l] == nums[l-1] and l < r:
                    l+= 1
    return res

In [67]:
#my solution, same, but i did not use elegant update like commented above
def threeSum(nums):
    if len(nums) < 3:
        return []
    lst = []
    nums.sort()
    for i in range(len(nums) - 2):
        if i > 0 and nums[i] == nums[i - 1]:
            continue
        temp = twoSum(nums[i + 1:], nums[i])
        if len(temp) > 0:
            lst += temp
    return lst
    
def twoSum(nums, first_num):
    temp_lst = []
    l, r = 0, len(nums) - 1
    while l < r:
        threeSum = first_num + nums[l] + nums[r]
        if threeSum == 0:
            temp_lst += [[first_num, nums[l], nums[r]]]
            l += 1
            while nums[l] == nums[l - 1] and l < r:
                l += 1
        elif threeSum > 0:
            r -= 1
            while nums[r] == nums[r + 1] and l < r:
                r -= 1
        else:
            l += 1
            while nums[l] == nums[l - 1] and l < r:
                l += 1
    return temp_lst

In [68]:
threeSum([-4,-1,-1,0,1,2])

[[-1, -1, 2], [-1, 0, 1]]